In [1]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

In [2]:
# ==========================================
# Part 1.1: Setup & Load CVs
# ==========================================
import os
import glob
from markitdown import MarkItDown


md = MarkItDown()
cv_files = sorted(glob.glob("CV_*.pdf"))

all_cvs = []
print(f"Found {len(cv_files)} CV files.")

for cv_path in cv_files:
    try:
        result = md.convert(cv_path)
        # 将文件名和文本内容存入列表
        all_cvs.append({
            "filename": os.path.basename(cv_path),
            "text": result.text_content
        })
        print(f"Successfully loaded: {cv_path}")
    except Exception as e:
        print(f"Error loading {cv_path}: {e}")

# 简单预览第一个 CV
if all_cvs:
    print(f"\n--- Preview of {all_cvs[0]['filename']} ---\n")
    print(all_cvs[0]['text'][:200] + "...")

Found 5 CV files.
Successfully loaded: CV_1.pdf
Successfully loaded: CV_2.pdf
Successfully loaded: CV_3.pdf
Successfully loaded: CV_4.pdf
Successfully loaded: CV_5.pdf

--- Preview of CV_1.pdf ---

John Smith

Marketing Professional

+ Singapore, Singapore (cid:209) Kowloon

Experience

Engineer, ByteDance

2020 – Present

• Worked in a fast-paced, global technology environment.

• Collaborated ...


In [6]:
# ==========================================
# Part 1.2: Verification Agent with QA Rules (Fixed Tool Call)
# ==========================================
import json
import asyncio
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage

# 1. 获取 API Key
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

# 2. 配置 LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=GEMINI_API_KEY
)

# 3. 连接 MCP Server
mcp_client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

async def verify_cv_logic(cv_text, filename):
    print(f"\n🔍 Processing {filename}...")

    # 获取工具并建立映射字典，方便后续调用
    try:
        tools = await mcp_client.get_tools()
        llm_with_tools = llm.bind_tools(tools)
        # 🔴 关键修复：把返回的工具列表转换成字典，方便通过名字查找
        tool_map = {tool.name: tool for tool in tools}
    except Exception as e:
        print(f"❌ Connection Error: Ensure the MCP server is running. {e}")
        return 0.0

    # --- Phase A: Investigation (搜集证据) ---
    investigation_prompt = f"""
    You are a background check expert. Your goal is to verify the candidate in this CV against the 'social_graph' database.

    CV Content:
    {cv_text}

    Strategy:
    1. Extract the candidate's Name.
    2. Search using 'search_linkedin_people' or 'search_facebook_users'.
       - IMPORTANT: If exact name search fails, try variations or set 'fuzzy=True'.
       - QA RULE: Do not give up if no exact match is found. Look for the *most similar* profile.
    3. Retrieve full details using 'get_linkedin_profile' (or Facebook equivalent) for the best match found.

    Output:
    Execute the necessary tool calls to find the profile.
    """

    messages = [HumanMessage(content=investigation_prompt)]
    tool_outputs = []

    for _ in range(5):
        ai_msg = await llm_with_tools.ainvoke(messages)
        messages.append(ai_msg)

        if not ai_msg.tool_calls:
            break

        for tool_call in ai_msg.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            print(f"   👉 Tool Call: {tool_name} args={tool_args}")

            # 🔴 关键修复：直接从 tool_map 拿出工具对象并调用
            if tool_name in tool_map:
                selected_tool = tool_map[tool_name]
                tool_result = await selected_tool.ainvoke(tool_args)
            else:
                tool_result = f"Error: Tool {tool_name} not found."

            result_str = str(tool_result)
            tool_outputs.append(f"Tool: {tool_name}, Result: {result_str}")

            messages.append(ToolMessage(content=result_str, tool_call_id=tool_call["id"]))

    # --- Phase B: Scoring (评分) ---
    scoring_prompt = f"""
    You are the final judge. Rate the truthfulness of the CV based on the gathered evidence.

    CV Content:
    {cv_text}

    Evidence (Social Media Search Results):
    {json.dumps(tool_outputs, indent=2)}

    JUDGMENT RULES (From QA Guidelines):
    1. **Internal Inconsistencies**: If the CV contradicts itself (e.g., overlapping dates), DO NOT reject it immediately. Instead, treat it as ground truth but LOWER the score appropriately.
    2. **No Exact Match**: If you cannot find an exact match in the database, do not default to 0. Find the *closest* profile. If the closest profile has differences, penalize the score but don't just say "not found".
    3. **Scoring Scale**:
       - 1.0: Perfect match.
       - 0.8-0.9: Match with minor typo or small date gap.
       - 0.4-0.7: Profile found but with exaggerated titles or major internal inconsistencies in CV.
       - 0.0-0.3: Complete fabrication or completely different person found.

    TASK:
    Output ONLY a single floating-point number between 0.0 and 1.0.
    """

    final_res = await llm.ainvoke([HumanMessage(content=scoring_prompt)])

    try:
        import re
        match = re.search(r"0\.\d+|1\.0|0|1", final_res.content)
        score = float(match.group()) if match else 0.0
    except:
        score = 0.0

    print(f"   ✅ Final Score for {filename}: {score}")
    return score

In [4]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }

In [5]:
# ==========================================
# Part 1.3: Run Batch & Evaluate
# ==========================================

predicted_scores = []

print("Starting verification process...\n")

if 'all_cvs' in locals() and all_cvs:
    for cv in all_cvs:
        score = await verify_cv_logic(cv['text'], cv['filename'])
        predicted_scores.append(score)

    print("\nAll Predicted Scores:", predicted_scores)
    # 假设 Ground Truth 是 [1, 1, 1, 0, 0]
    groundtruth = [1, 1, 1, 0, 0]
    result = evaluate(predicted_scores, groundtruth)
    print("\nEvaluation Report:")
    print(result)

else:
    print("⚠️ 未找到 all_cvs 变量，请先运行 Block 1.1 加载 PDF。")

Starting verification process...


🔍 Processing CV_1.pdf...
   👉 Tool Call: search_linkedin_people args={'q': 'John Smith', 'location': 'Singapore'}
   👉 Tool Call: get_linkedin_profile args={'person_id': 9}
   ✅ Final Score for CV_1.pdf: 0.6

🔍 Processing CV_2.pdf...
   👉 Tool Call: search_linkedin_people args={'q': 'Minh Pham', 'location': 'Hong Kong'}
   👉 Tool Call: search_linkedin_people args={'location': 'Beijing, China', 'q': 'Minh Pham'}
   👉 Tool Call: search_linkedin_people args={'q': 'Minh Pham', 'fuzzy': True}
   👉 Tool Call: get_linkedin_profile args={'person_id': 47}
   ✅ Final Score for CV_2.pdf: 1.0

🔍 Processing CV_3.pdf...
   👉 Tool Call: search_linkedin_people args={'q': 'Wei Zhang', 'limit': 1}
   👉 Tool Call: get_linkedin_profile args={'person_id': 24}
   👉 Tool Call: search_linkedin_people args={'location': 'Munich', 'limit': 1, 'fuzzy': True, 'q': 'Wei Zhang'}
   👉 Tool Call: get_linkedin_profile args={'person_id': 97}
   ✅ Final Score for CV_3.pdf: 0.5

🔍 Proces